<a href="https://colab.research.google.com/github/laraimad/Machine-learning-assignment-/blob/main/maleria(r2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path='/content/drive/MyDrive/mldata0.xlsx'
df = pd.read_excel(file_path, sheet_name=7)
df

/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 19 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")
/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 12 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")


,HealthRegion,No.Examined,PositiveCases,SlidePositivity(%),P.falciparum(Malignant),P. vivax and Povale (Benign Tertiary),P. malariae (Benign Quartan),Mixed,Introdused,Imported
0,Central Region,20324,172,0.84629,86,80,3,3,0,172
1,Westren Region,175569,761,8.65000,394,328,11,28,0,761
2,Eastren Region,221027,322,7.68000,172,148,2,0,0,322
3,Southren Region,223311,2951,3.84000,2383,555,5,8,274,2677
4,Northren Region,124591,113,7.32000,44,68,1,0,0,108


In [ ]:
df.columns

Index(['HealthRegion', 'No.Examined', 'PositiveCases', 'SlidePositivity(%)',
       'P.falciparum(Malignant)', 'P. vivax and Povale (Benign Tertiary)',
       'P. malariae (Benign Quartan)', 'Mixed', 'Introdused', 'Imported'],
      dtype='object')

In [ ]:
# Convert 'HealthRegion' to numerical using Label Encoding
# Ensure this is applied to X_selected directly
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X_selected['HealthRegion'] = label_encoder.fit_transform(X_selected['HealthRegion'])

In [ ]:
# Train-test split again
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# Use regression models instead
model = DecisionTreeRegressor(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


Mean Absolute Error: 1.0


In [ ]:
df_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(df_results.head(10))


   Actual  Predicted
1       1        0.0


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 10, 20, None],  # Try different depths
    'min_samples_split': [2, 5, 10],   # Control when the tree splits
    'min_samples_leaf': [1, 2, 5]      # Minimum samples per leaf
}

grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=2)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

# Train with best parameters
best_model = DecisionTreeRegressor(**grid_search.best_params_)
best_model.fit(X_train, y_train)

# Predict again
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Improved MAE:", mae)


Best parameters: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
Improved MAE: 0.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Random Forest MAE:", mae)


Random Forest MAE: 0.54


In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Linear Regression MAE:", mae)


Linear Regression MAE: 0.22105544890829865


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

importances = rf.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


HealthRegion: 0.1000
No.Examined: 0.1000
SlidePositivity(%): 0.0667
P.falciparum(Malignant): 0.0889
P. vivax and Povale (Benign Tertiary): 0.2222
P. malariae (Benign Quartan): 0.1889
Mixed: 0.0222
Introdused: 0.0778
Imported: 0.1333


In [ ]:


# Drop weak features first
X_selected = X.drop(columns=['Mixed', 'Introdused'])

# Ensure 'HealthRegion' is numerical
label_encoder = LabelEncoder()
X_selected['HealthRegion'] = label_encoder.fit_transform(X_selected['HealthRegion'])

# Now we split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42
)

# Train the best model (Linear Regression)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict and evaluate
y_pred = lr.predict(X_test)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

print("Final Linear Regression MAE:", mae)


Final Linear Regression MAE: 1.0549478207837084


In [ ]:


# Create a DataFrame to compare actual and predicted values
df_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Display the first 10 rows
print(df_results.head(10))


  Actual  Predicted
1      2   3.054948
